# Discovery of Musical Patterns and Structures using Mathematical Morphology

<div style="display: flex; justify-content: space-between; align-items: center; padding: 20px 0;">
    <img src="https://imgs.search.brave.com/xY8R_8l4B2YioiaJAn-_C_rdzuVm0GxAbIF6mx9uglE/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly9pcm1p/YXBwLnVuaXN0cmEu/ZnIvd2Vic2l0ZXMv/SVRJLUlSTUlBcHAv/TG9nb3MvbG9nb19p/cm1hLnBuZw" style="height: 80px;">
    <img src="https://imgs.search.brave.com/abpAYumpwZ7KG6ngCp5KWlZNqAdbOjBuAe13-lX2awg/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly93d3cu/dW5pc3RyYS5mci9m/aWxlYWRtaW4vdGVt/cGxhdGVzL3VuaXN0/cmEtdjMvdGVtcGxh/dGVzL2ltYWdlcy93/ZWItZXQtYXNzb2Np/ZS1OQi5wbmc" style="height: 80px;">
</div>

**Author:** Mathys DANIEL

**Based on the work of:** Paul LASCABETTES (Thesis: [https://hal.science/tel-04480227](https://hal.science/tel-04480227))

## Project Description

**Objective:** To develop a fully automatic program that captures musical patterns and deduces time signatures for individual tracks and instruments using exclusively topological tools, achieving greater computational efficiency than state-of-the-art methods.

**Methodology:** We will leverage mathematical morphology to analyze the inherent structure of musical data, identifying recurring motifs and rhythmic variations.

**Expected Outcome:** To demonstrate the program’s ability to handle challenging rhythmic structures and extract meaningful musical information with minimal computational overhead.


# Import

In [ ]:
# Install with proper versions
!python -m pip install -r requirements.txt

In [ ]:
import os
import re
import time
import math
from math import isclose, modf
from fractions import Fraction
from collections import defaultdict
from pathlib import Path
from typing import List, Tuple, Dict, Callable

# Scientific stack
import numpy as np
import pandas as pd

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.ticker as mticker
from matplotlib.ticker import MultipleLocator
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches

# MIDI processing
import pretty_midi


# Topological Tools

This section defines some topological tools that are useful for our algorithms.

In [ ]:
def erosion(X, S):
    """
    Compute the erosion of X by structuring element S, with S centered at the origin.
    """
    # Get the first point of S to determine the translation vector
    s0, s1 = S[0]

    # Translate S to be centered at the origin
    S_centered = [(so - s0, sp - s1) for (so, sp) in S]

    eroded = list()

    X = set(X)

    for (o, p) in X:
        # Check if translated structuring element is included in X
        if all((o + so, p + sp) in X for (so, sp) in S_centered):
            eroded.append((o, p))

    return eroded

def dilation(X, S):
    """
    Compute the dilation of set X by structuring element S.
    """
    dilated = list()

    X = set(X)

    # Get the first point of S to determine the translation vector
    s0, s1 = S[0]

    # Translate S to be centered at the origin
    S_centered = [(so - s0, sp - s1) for (so, sp) in S]

    for (o, p) in X:
        for (so, sp) in S_centered:
            dilated.append((o + so, p + sp))

    return dilated

# opening (erosion then dilation)
def opening(X, S):
    """
    Compute the opening of set X by structuring element S.
    """
    return dilation(erosion(X, S), S)

# morphological closing of a set X (dilation then erosion)
def closing(X, S):
    """
    Compute the closing of set X by structuring element S.
    """
    return erosion(dilation(X, S), S)

# Metric Recognition

This section defines the algorithm that detects the patterns in a given piece of music.

In [ ]:
def windowing(P, L):
    """
    Splits the pattern P into multiple sub-patterns of exact length L.
    - Ensures each window starts at a valid onset.
    - Prevents empty windows.
    """
    if not P:
      return []

    P = sorted(P, key=lambda x: x[0])  # Sort by onset
    windowed_patterns = []
    i = 0  # Index to track starting point

    while i < len(P):
        t1 = P[i][0]  # Base onset for this window
        window = [p for p in P if t1 <= p[0] < t1 + L]

        if not window:  # Skip empty windows
            i += 1
            continue

        windowed_patterns.append(window)

        # Move to next onset after the current window
        while i < len(P) and P[i][0] < t1 + L:
            i += 1

    return windowed_patterns

def filtering(P_list, P_full, L, Nmin=4):
    """
    Filter and merge the subpatterns that repeats periodically inside the previously found patterns
    """
    if not P_list:
        return []

    P_to_test = [tuple(map(tuple, Pj)) for Pj in P_list]
    P_output = []
    P_all = P_to_test.copy()
    P_to_test = sorted(P_to_test, key=lambda pat: min(p[0] for p in pat))

    while P_to_test:
        Pj = P_to_test.pop(0)
        Pj_list = list(Pj)
        onset_j = min(p[0] for p in Pj_list)
        opening_Pj = opening(P_full, Pj_list)

        Nj = Nmin
        absorbed = []

        # This list tracks all valid successive window onsets relative to Pj
        reference_onsets = [onset_j]

        for Pk in P_all:
            if Pk == Pj or Pk not in P_to_test:
                continue

            Pk_list = list(Pk)
            onset_k = min(p[0] for p in Pk_list)

            if any(p in opening_Pj for p in Pk_list):
                # Check if onset_k is in a window immediately after one of the reference windows
                window_diff_ok = any(round((onset_k - ref_onset) / L) == 1 for ref_onset in reference_onsets)

                if window_diff_ok:
                    Nj += 1
                    if all(p in opening_Pj for p in Pk_list):
                        absorbed.append(Pk)
                        reference_onsets.append(onset_k)  # Extend the chain!

        for Pk in absorbed:
            if Pk in P_to_test:
                P_to_test.remove(Pk)

        P_output.append((Pj_list, Nj))

    return P_output


def pattern_detection(X, Lmax = 20, Nmin=4):
    """
    Extracts translation-invariant patterns from input list X for various L values.
    - Applies erosion to detect patterns.
    - Uses windowing to extract patterns of exact length L.
    - Uses filtering to refine extracted patterns.
    """
    found_patterns = []
    L_values = np.arange(2, Lmax, 0.25)

    for L in L_values:
        # Step 1: Define structuring element O_L as a list
        O_L = [(n*L, 0) for n in range(Nmin)]

        # Step 2: Apply erosion to find patterns
        P = erosion(X, O_L)

        # Step 3: Windowing to get patterns of exact length L
        P1_to_Pn = windowing(P, L)

        # Step 4: Filtering to refine patterns
        filtered_patterns = filtering(P1_to_Pn, P, L, Nmin)

        # Step 5: Store results
        for P_prime, N in filtered_patterns:
            found_patterns.append((P_prime, N, float(L)))

    return found_patterns

# Metric visualization

### Ground Truth

This section extracts time signature from dataset info

In [ ]:
def _parse_ts_string(ts_string):
    """
    '7/8 @ 0.000 beats; 6/4 @ 406.000 beats'
        -> [{'beat':0.0,'num':7,'den':8}, …]
    """
    if not ts_string:
        return []
    events = []
    for chunk in ts_string.split(';'):
        m = re.match(r'\s*(\d+)\s*/\s*(\d+)\s*@\s*([0-9.]+)', chunk)
        if m:
            num, den, beat = int(m[1]), int(m[2]), float(m[3])
            events.append({'beat': beat, 'num': num, 'den': den})
    return sorted(events, key=lambda d: d['beat'])
# ------------------------------------------------- ①

def _format_time_signature_in_beats(ts_obj, pm):
    """
    Returns the time signature in beats.
    """
    # Convert absolute seconds → ticks → beats
    beat_idx = pm.time_to_tick(ts_obj.time) / pm.resolution
    return f"{ts_obj.numerator}/{ts_obj.denominator} @ {beat_idx:.3f} beats"


def check_time_signature_presence(folder_path, show_table = False):
    """
    Scan every MIDI file in folder_path, report whether it contains
    Time-Signature events, how many, and list them.
    """
    rows = []

    for fname in os.listdir(folder_path):
        if not fname.lower().endswith((".mid", ".midi")):
            continue

        path = os.path.join(folder_path, fname)
        try:
            pm = pretty_midi.PrettyMIDI(path)
            ts_changes = pm.time_signature_changes                   # list[TimeSignature]
            n_ts       = len(ts_changes)

            # ---------- tempo information ---------------------------
            tempo_changes, tempi = pm.get_tempo_changes()
            bpm = float(tempi[0]) if len(tempi) else 120.0

            ts_strs = [_format_time_signature_in_beats(ts, pm) for ts in ts_changes]    # turn each TimeSignature into a readable string (in beats)


            rows.append({
                "midi_file"                 : fname,
                "bpm"                       : bpm,
                "has_time_signature"        : bool(n_ts),
                "num_time_signature_events" : n_ts,
                "time_signatures"           : "; ".join(ts_strs) if ts_strs else ""
            })
        except Exception as e:
            print(f"Could not parse {fname}: {e}")

    df = pd.DataFrame(rows)

    # Print the table with all columns
    if show_table:
      if not df.empty:
            print(df.to_string(index=False))
      else:
          print("No MIDI files found.")

      # Global summary
      if df["has_time_signature"].any():
          print("\n✓  At least one file contains explicit time-signature data.")
      else:
          print("\n✗  No file in the folder contains any time-signature events.")

    return df

### Plot sections

This section defines all the tools and apply them to visualize the metric construction of a given piece of music, using the algorithm previously defined.

In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
#  0) Figure parameters definition
# ──────────────────────────────────────────────────────────────────────────────
mpl.rcParams.update({
    # --- fonts -----------------------------------------------------
    'font.family'       : 'serif',
    'font.serif'        : ['Computer Modern Roman', 'cmr10'],
    'mathtext.fontset'  : 'cm',
    'font.size'         : 11,          # base text size
    'axes.titlesize'    : 14,          # title → a bit larger
    'axes.titleweight'  : 'bold',

    # --- axes & spines --------------------------------------------
    'axes.edgecolor'    : '#333333',
    'axes.linewidth'    : .8,
    'axes.grid'         : False,       # we enable it manually, x–only
    'xtick.direction'   : 'out',
    'ytick.direction'   : 'out',

    # --- legend ----------------------------------------------------
    'legend.frameon'    : False,
    'legend.fontsize'   : 9,

    # --- figure / saving ------------------------------------------
    'figure.dpi'        : 150,
    'savefig.dpi'       : 300,
    'figure.autolayout' : True         # keeps labels from colliding
})

# ---------- SECTIONS COLOR MAPPING ---------------------------
PRESET_BASE_COLORS = {
     2 : "#bcbd22",   # olive (close to green, because it's musically close to 4)
     3 : "#FF0000",   # red
     4 : "#2ca02c",   # green
     5 : "#ff7f0e",   # orange
     7 : "#6a3d9a",   # purple
     9 : "#17becf",   # teal (separated from 3, because 9 is often not used as 3x3 but as a whole 9)
    11 : "#8c564b",   # brown
    13 : "#E30B5C",   # pink
    17 : "#008080",   # teal
}

# ──────────────────────────────────────────────────────────────────────────────
#  1)  Low-level numeric helpers
# ──────────────────────────────────────────────────────────────────────────────

def group_multiplicative_bases(Ls: List[float]) -> List[List[float]]:
    """
    Group L values by shared multiplicative base.

    If multiple base candidates are valid (i.e., L is divisible by several others),
    we assign it to the group with the **largest** such base.

    Returns
    -------
    base_groups : list of groups, each group being a list of Ls.
    """
    base_groups = []
    base_lookup = {}  # maps base -> group

    for L in sorted(Ls):
        best_base = None
        for base in base_lookup:
            ratio = L / base
            ratio_inv = base / L
            if (isclose(ratio, round(ratio), abs_tol=1e-8) and ratio >= 1) or \
               (isclose(ratio_inv, round(ratio_inv), abs_tol=1e-8) and ratio_inv >= 1):
                if best_base is None or base > best_base:
                    best_base = base
        if best_base is not None:
            base_lookup[best_base].append(L)
        else:
            base_lookup[L] = [L]

    return list(base_lookup.values())


def assign_vertical_layers(x_positions: List[float], min_gap: float) -> List[int]:
    """
    Greedy layer assignment so that labels (placed at each x_positions)
    do not overlap horizontally by less than min_gap.

    Returns a list `layer_index`  (same length as x_positions).
    """
    layers, label_layers = [], []          # running accumulation
    for x in x_positions:
        # Try to squeeze into an existing layer
        for i, layer in enumerate(layers):
            if all(abs(x - lx) > min_gap for lx in layer):
                layer.append(x)
                label_layers.append(i)
                break
        else:
            # No layer was available → create a new one
            layers.append([x])
            label_layers.append(len(layers) - 1)
    return label_layers


# ──────────────────────────────────────────────────────────────────────────────
#  2)  Pattern / compactness computation
# ──────────────────────────────────────────────────────────────────────────────
def compute_compactness_segments_for_instrument(
        points: List[Tuple[float, int]],
        pattern_detection: Callable,
        dilation: Callable[[list, list], list],
        global_id_start: int,
        Lmax: int
) -> Tuple[List[Dict], Dict[int, Dict], int]:
    """
    For one instrument track:
    • Call pattern_detection to get repeating patterns.
    • Dilate each pattern (so repetitions overlay each other).
    • Compute a 'compactness' score.
    • Assign a globally unique band ID starting from global_id_start.

    Returns
    -------
    bands : list of filtered bands (dicts)
    band_map : dict of {id → band}
    next_id : next available global ID
    """
    patterns = pattern_detection(points, Lmax=Lmax, Nmin=4)
    bands = []
    band_map = {}
    points_set = set(map(tuple, points))
    current_id = global_id_start

    for P_i, N_i, L_i in patterns:
        if not P_i:
            continue

        O_L_N = [(n * L_i, 0) for n in range(N_i)]
        dilated_i = dilation(P_i, O_L_N)
        if not dilated_i:
            continue

        dilated_set = set(map(tuple, dilated_i))
        x_min = min(o for o, _ in dilated_i)
        x_max = max(o for o, _ in dilated_i)
        pts_in_range = [p for p in points if x_min <= p[0] <= x_max]
        pertes = len(pts_in_range) - len(dilated_set)
        compactness = len(pts_in_range) / (len(P_i) + 2 + pertes)

        band = dict(
            id=current_id,
            L=L_i, N=N_i, compactness=compactness,
            pattern_len=len(P_i),
            min_on=x_min, max_on=x_max,
            dilated_set=dilated_set,
            original_points_set=points_set
        )
        bands.append(band)
        band_map[current_id] = band
        current_id += 1

    filtered = refine_overlapping_segments(bands)
    return filtered, band_map, current_id


def refine_overlapping_segments(
        bands: List[Dict],
        max_iterations: int = 5 # chosen arbitrarily for computational purposes
) -> List[Dict]:
    """
    Resolve overlaps between candidate bands, keeping the most compact one.

    Compactness is re-evaluated after removing shared points; the process is
    iterated until no further changes occur or max_iterations hits.
    """
    bands.sort(key=lambda b: b['min_on'])
    for _ in range(max_iterations):
        changed = False
        for i, b1 in enumerate(bands):
            for j in range(i + 1, len(bands)):
                b2 = bands[j]
                if b2['min_on'] > b1['max_on']:   # no horizontal overlap → break
                    break
                if b1['max_on'] >= b2['min_on']:  # overlap exists
                    winner, loser = (b1, b2) if b1['compactness'] > b2['compactness'] else (b2, b1)
                    overlap = loser['dilated_set'] & winner['dilated_set']
                    if overlap:
                        # Remove shared points from the loser
                        loser['dilated_set'] -= overlap
                        if not loser['dilated_set']:
                            loser['compactness'] = float('inf')
                            changed = True

                        else:
                            x_min = min(p[0] for p in loser['dilated_set'])
                            x_max = max(p[0] for p in loser['dilated_set'])

                            pts_in_range = [p for p in loser['original_points_set']
                                            if x_min <= p[0] <= x_max]
                            pertes = len(pts_in_range) - len(loser['dilated_set'])
                            if  (loser['pattern_len'] + 2 + pertes) <= 0:
                              loser['compactness'] = float('inf')
                              continue
                            # Update the loser's range and compactness
                            loser.update(min_on=x_min,
                                        max_on=x_max,
                                        compactness=len(pts_in_range) / (loser['pattern_len'] + 2 + pertes))
                            changed = True
        # Purge invalidated losers
        bands = [b for b in bands if b['compactness'] != float('inf')]
        if not changed:
            break
    return bands

def compute_compactness_segments_for_all_instruments(
        track_instruments: Dict[str, list],
        pattern_detection: Callable,
        dilation: Callable,
        Lmax: int
) -> Tuple[Dict[str, List[Dict]], Dict[int, Dict]]:
    """
    For all instrument tracks:
    • Call compute_compactness_segments_for_instrument with global IDs.
    • Collect filtered bands per instrument.
    • Return global band map and per-instrument bands.

    Returns
    -------
    all_bands : dict[str → list of band dicts]
    band_map  : dict[id → band dict]
    """
    all_bands = {}
    band_map = {}
    global_id = 0

    for inst, points in track_instruments.items():
        filtered, band_dict, global_id = compute_compactness_segments_for_instrument(points, pattern_detection, dilation, global_id, Lmax)
        all_bands[inst] = filtered
        band_map.update(band_dict)

    return all_bands, band_map

def extract_best_compactness_and_merge_intervals(
        bands: List[Dict]
) -> Tuple[List[Dict], List[Dict]]:
    """
    Extract the best-scoring (biggest compactness) segment for every elementary
    sub-interval found in bands.

    Returns
    -------
    exhaustive_intervals : List[Dict]
        Complete list of all non-overlapping intervals (one per elementary segment)
    merged_intervals : List[Dict]
        Merged contiguous segments for display purposes only
    """
    # ➊  Obtain all cut points
    cps = sorted({pt for b in bands for pt in (b['min_on'], b['max_on'])})
    exhaustive_intervals = []

    # ➋  For each elementary (a, b) choose the best band covering it
    for a, b in zip(cps, cps[1:]):
        cand = [x for x in bands if a < x['max_on'] and b > x['min_on']]
        if cand:
            best = max(cand, key=lambda x: (x['compactness'],  x['N'],  x['L']))
            exhaustive_intervals.append({'band_id':  best['id'],
                              'L': best['L'],
                              'compactness': best['compactness'],
                              'a': a, 'b': b})

    # Keep only merged segments whose length ≥ one full L
    exhaustive_intervals = [s for s in exhaustive_intervals if (s['b'] - s['a']) >= s['L']]

    # ➌  Create merged version for display (merge consecutive segments with same L)
    merged_intervals = []

    # Sort segments by start position
    sorted_segments = sorted(exhaustive_intervals, key=lambda x: x['a'])

    for seg in sorted_segments:
        # Check if we can merge with the last segment in merged_intervals
        if (merged_intervals and
            isclose(seg['L'], merged_intervals[-1]['L']) and
            # Check if there's no other segment between the current and the last merged one
            not any(other_seg['b'] > merged_intervals[-1]['b'] and other_seg['a'] < seg['a']
                   for other_seg in sorted_segments
                   if not isclose(other_seg['L'], seg['L']))):

            # Extend the last merged segment
            merged_intervals[-1]['b'] = seg['b']
            # Update compactness to the maximum of the merged segments
            merged_intervals[-1]['compactness'] = max(merged_intervals[-1]['compactness'], seg['compactness'])
        else:
            # Create new merged segment (copy of current segment)
            merged_intervals.append(seg.copy())

    return exhaustive_intervals, merged_intervals


def compute_global_compactness(track_instruments: Dict[str, list],
                               bands_by_inst: Dict[str, List[Dict]]) -> float:
    """
    Global score

        GC = |P_tot| /
             ( Σ_patterns (len(pattern) + 2)  +  |misses| )

    • P_tot      : set of every original (time, pitch) point
    • len(pattern): stored in each band as  b['pattern_len']
    • misses     : original points that do **not** belong to any
                   dilated_set of the retained patterns
    """
    # ➊  All original points ---------------------------------------
    all_points = {
        tuple(p)
        for pts in track_instruments.values()
        for p   in pts
    }

    # ➋  Union of all dilated points and the pattern-cost term -----
    covered_points   = set()
    penalty_patterns = 0          # Σ (len(pattern) + 2)
    n_motifs = 0

    for bands in bands_by_inst.values():
        for b in bands:
            covered_points |= b['dilated_set']
            penalty_patterns += (b['pattern_len']*2 + 2)
            n_motifs += 1

    # ➌  Points not covered by any pattern -------------------------
    misses = all_points - covered_points

    # ➍  Final compactness -----------------------------------------
    print(f"Pourcentage de points perdus : {100 * len(misses) / len(all_points):.2f} %")
    print("Nombre de motifs:", n_motifs)

    return len(all_points)*2 / (penalty_patterns + len(misses)*2)

# ──────────────────────────────────────────────────────────────────────────────
#  3)  Colour assignment for metric groups
# ──────────────────────────────────────────────────────────────────────────────
def assign_color_groups_by_multiplicative_base(all_Ls: List[float]):
    """
    Create a colour map for each encountered L value.

    • All lengths are grouped by the smallest integer 'base' such that
      they differ by an integer ratio; each group gets its own colour.

    If a base is not found in the mapping dictionary (PRESET_BASE_COLORS)
    and has no multiplicative relation to one that is, it gets default grey.
    """

    # 1) Build multiplicative groups
    base_groups = group_multiplicative_bases(all_Ls)

    # 2) Label each group by its smallest integer (if any)
    base_labels = {}
    for group in base_groups:
        int_members = [x for x in group if isclose(x, round(x), abs_tol=1e-8)]
        label = int(min(int_members)) if int_members else min(group)
        for L in group:
            base_labels[L] = label

    # Special case for L=2 (because it's difficult to handle, half the integer values would be in its base)
    if any(isclose(L, 2.0, abs_tol=1e-8) for L in all_Ls):
        base_labels[2.0] = 2

    # 3) Color each base label
    color_for_base = {
        b: mcolors.to_rgba(c) for b, c in PRESET_BASE_COLORS.items()
    }

    known_bases = set(PRESET_BASE_COLORS.keys()) - {2}

    def find_related_base(base, knowns):
        for known in knowns:
            ratio = base / known
            ratio_inv = known / base
            if (isclose(ratio, round(ratio), abs_tol=1e-8) and ratio >= 1) or \
               (isclose(ratio_inv, round(ratio_inv), abs_tol=1e-8) and ratio_inv >= 1):
                return known
        return None

    for base in sorted({base_labels[L] for L in base_labels}):
        if base in color_for_base:
            continue
        related = find_related_base(base, known_bases)
        if related and related in color_for_base:           # in a mapped base
            color_for_base[base] = color_for_base[related]
        else:                                               # not in a mapped base
            color_for_base[base] = (0.6, 0.6, 0.6, 0.5)  # Default grey

    # 4) Final L → RGBA assignment
    color_for_L = {}
    for L in all_Ls:
        base = base_labels.get(L, L)
        color_for_L[L] = color_for_base[base]

    return color_for_L, base_labels, color_for_base


# ──────────────────────────────────────────────────────────────────────────────
#  4)  Plotting helpers
# ──────────────────────────────────────────────────────────────────────────────
def plot_instrument_onsets(ax, track_instruments, instrument_to_y):
    """
    Scatter-plot every individual onset for every instrument.

    Parameters
    ----------
    ax : matplotlib-axes
        The axes on which everything is drawn.

    track_instruments : dict[str, list[(float,int)]]
        Maps an instrument-name to a list of tuples (onsetBeat, midiPitch).

    instrument_to_y : dict[str, int]
        Pre-computed mapping  instrument → y-row index.
    """
    for inst, pts in track_instruments.items():
        y = instrument_to_y[inst]
        ax.scatter(
            [float(o) for o, _ in pts],   # x-coordinates (beats), but final output will be in seconds
            [y] * len(pts),               # y-coordinates (fixed row)
            s=10, color='black', alpha=.3
        )


def plot_segments_with_stacked_labels(ax, intervals_by_inst,
                                      instrument_to_y, color_for_L):
    """
    Draw the coloured metric segments (rectangles) and print L values
    stacked above them while avoiding label collisions.

    Parameters
    ----------
    intervals_by_inst : dict[str, list[dict]]
        Output of extract_and_merge_best_compactness_intervals per instrument.

    color_for_L : dict[float, rgba]
        Mapping from every observed L value to its display colour.
    """
    vertical_spacing = .18                       # distance between stacked labels

    for inst, merged in intervals_by_inst.items():
        y = instrument_to_y[inst]
        if not merged:
            continue

        # ➊  Compute label centres and layer organisation (collision-avoiding)
        xcent = [(s['a'] + s['b']) / 2 for s in merged]
        order = sorted(range(len(xcent)), key=lambda i: xcent[i])
        min_gap = .03 * (ax.get_xlim()[1] - ax.get_xlim()[0] or 1)
        layers = assign_vertical_layers([xcent[i] for i in order], min_gap)

        layer_idx = [0] * len(merged)
        for idx, layer in zip(order, layers):
            layer_idx[idx] = layer

        # ➋  Actually draw rectangles + labels
        for i, s in enumerate(merged):
            L = s['L']
            if L == int(L):
                L_str = str(int(L))
            elif L * 10 == int(L * 10):  # ex: 5.5
                L_str = f"{L:.1f}"
            else:
                L_str = f"{L:.2f}"  # fallback for unusual cases

            ax.add_patch(Rectangle((
                        s['a'], y - .3),           # (x, y) lower-left
                        s['b'] - s['a'], .6,        # width / height
                        facecolor=color_for_L.get(s['L'], (0, 0, 0, .1)),
                        edgecolor=color_for_L.get(s['L'], (0, 0, 0, .1)),
                        alpha=.3))

            ax.text((s['a'] + s['b']) / 2,
                    y + .15 + layer_idx[i] * vertical_spacing,
                    L_str,
                    ha='center', va='bottom',
                    fontweight='bold', fontsize=10, fontfamily='DejaVu Sans')


# ──────────────────────────────────────────────────────────────────────────────
#  5)  High-level driver: MAIN PLOT (plot_metric_sections_by_best_compactness)
# ──────────────────────────────────────────────────────────────────────────────

def plot_metric_sections_by_best_compactness(
        subset_data: dict,
        track_name: str,
        pattern_detection, dilation,
        ground_truth_df,
        Lmax=20):
    """
    Master routine that puts everything together:

    • Extract onsets for track_name from subset_data.
    • Compute metric candidates, select best by compactness.
    • Draw one horizontal strip per instrument plus an extra
      'Database Ground Truth' strip.
    • Overlay colour-coded rectangles and labels.
    • Add a legend grouping L values by their multiplicative base.

    Parameters
    ----------
    subset_data : dict
        {instrument_name → [(onsetBeat, midiPitch), …]}

    track_name : str
        Name (suffix) of the MIDI file we want to visualise.

    algo, dilation : callables
        User-supplied routines used earlier for pattern detection / dilation.

    ground_truth_df : pandas.DataFrame
        Data-frame holding the reference time-signature annotations.

    Lmax : int, optional (default=20)
        Maximum metric length to consider when searching for patterns.
    """

    # ------------------------------------------------------------------
    # ➊  Collect all onsets belonging to that MIDI file
    # ------------------------------------------------------------------
    track_instruments = {k: v for k, v in subset_data.items()
                         if k.startswith(track_name)}
    if not track_instruments:
        print(f"No data for {track_name}")
        return

    # Row mapping (instrument → y-coordinate on the plot)
    instruments = sorted(track_instruments)
    instrument_to_y = {inst: i for i, inst in enumerate(instruments)}

    # ------------------------------------------------------------------
    # ➋  Analyse compactness for every instrument
    # ------------------------------------------------------------------
    all_bands, band_map = compute_compactness_segments_for_all_instruments(
        track_instruments, pattern_detection, dilation, Lmax)

    intervals_by_inst = {}
    merged_intervals_by_inst = {}  # Only for display

    for inst, band in all_bands.items():
        exhaustive_intervals, merged_intervals = extract_best_compactness_and_merge_intervals(band)
        if exhaustive_intervals:  # Keep all exhaustive intervals
            intervals_by_inst[inst] = exhaustive_intervals
            merged_intervals_by_inst[inst] = merged_intervals  # For display

    used_ids = {
        seg['band_id']
        for segs in intervals_by_inst.values()
        for seg  in segs}

    display_bands = {
        inst: [band_map[seg['band_id']] for seg in segs]
        for inst, segs in intervals_by_inst.items()}

    # Colour bookkeeping
    all_L = {s['L'] for v in merged_intervals_by_inst.values() for s in v}
    color_for_L, base_for_L, color_for_base = assign_color_groups_by_multiplicative_base(all_L)

    # ------------------------------------------------------------------
    # ➌  Append an extra staff for ground-truth signatures
    # ------------------------------------------------------------------
    gt_name = "Database Ground Truth"
    instruments.append(gt_name)
    instrument_to_y = {inst: i for i, inst in enumerate(instruments)}
    gt_y = instrument_to_y[gt_name]

    # ------------------------------------------------------------------
    # ➍  Prepare figure & draw machine-estimated segments
    # ------------------------------------------------------------------
    plt.figure(figsize=(16, 1 + .8 * len(instruments)))
    ax = plt.gca()

    plot_instrument_onsets(ax, track_instruments, instrument_to_y)

    plot_segments_with_stacked_labels(ax, merged_intervals_by_inst,
                                      instrument_to_y, color_for_L)

    # ------------------------------------------------------------------
    # ➎  Draw ground-truth rectangles (if any) on the bottom staff
    # ------------------------------------------------------------------
    row = ground_truth_df[ground_truth_df['midi_file']
                          .str.endswith(track_name, na=False)]

    if not row.empty:
        events = _parse_ts_string(row.iloc[0]['time_signatures'])
        if events:
            x_max = ax.get_xlim()[1]

            # Compute centre-points and layer indices for GT labels
            gt_centers = []
            for i, ev in enumerate(events):
                x0 = ev['beat']
                x1 = events[i + 1]['beat'] if i + 1 < len(events) else x_max
                gt_centers.append((x0 + x1) / 2)

            gt_layers = assign_vertical_layers(
                gt_centers, min_gap=0.03 * (x_max - 0))

            # Actual drawing loop
            L_gt_list = []
            for i, ev in enumerate(events):
                L_gt = ev['num']
                L_gt_list.append(L_gt)
                x0 = ev['beat']
                x1 = events[i + 1]['beat'] if i + 1 < len(events) else x_max
                color_for_L_gt, _, _ = assign_color_groups_by_multiplicative_base(L_gt_list)
                color = color_for_L_gt.get(L_gt, (0, 0, 0, 0.2))

                # Background rectangle
                ax.add_patch(Rectangle((x0, gt_y - .3), x1 - x0, .6,
                                       facecolor=color, edgecolor= color,
                                       alpha=.3, linewidth=.8, zorder=5))

                # Time-signature label (e.g. "7/8")
                y_offset = -0.10 + gt_layers[i] * .18
                ax.text((x0 + x1) / 2, gt_y + y_offset,
                        f"{ev['num']}/{ev['den']}",
                        ha='center', va='bottom',
                        fontsize=12, fontweight='bold', fontfamily='DejaVu Sans',
                        color=color, zorder=6)

    # ------------------------------------------------------------------
    # ➏  Cosmetics: axes, legend, layout
    # ------------------------------------------------------------------
    ax.set_yticks(list(instrument_to_y.values()))
    ax.set_yticklabels([k.split('_')[-1] for k in instruments])

    # ------ Convert the beats axis in seconds for visibility purpose -------
    track_row = ground_truth_df.loc[
        ground_truth_df['midi_file'].str.endswith(track_name, na=False)]

    bpm = float(track_row['bpm'].iloc[0]) if not track_row.empty else 120.0
    beat_to_sec = 60 / bpm
    ax.set_xlabel("Temps (s)")

    ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda beat, pos: f"{beat*beat_to_sec:.0f}"))

    sec_step   = 60
    beat_step  = sec_step / beat_to_sec     # beats per 60 s  (== bpm)
    ax.xaxis.set_major_locator(mticker.MultipleLocator(beat_step))     # major ticks every 60s
    ax.xaxis.set_minor_locator(mticker.MultipleLocator(beat_step/2))   # minor ticks every 30s
    display_name = track_name.removesuffix('.mid')
    ax.set_title(f"{display_name}")
    ax.grid(True, axis='x', ls='--', alpha=.5)
    ax.set_xlim(left=0)
    ax.set_ylim(-.5, gt_y + .5)

    # ---- Legend showing multiplicative bases -------------------------

    legend_patches = [mpatches.Patch(color=color_for_base[base],
                      label=f"{base:g}",
                       alpha=0.3)
        for base in sorted(PRESET_BASE_COLORS.keys())]

    DEFAULT_OTHER_COLOR = (0.6, 0.6, 0.6, .5)
    default_patch = mpatches.Patch(color=DEFAULT_OTHER_COLOR,label="Autres")
    legend_patches.append(default_patch)

    if legend_patches:
        ncol = min(len(legend_patches), 8)
        ax.legend(handles=legend_patches,
                  title="Signature rythmique associee",
                  ncol=ncol,
                  loc='lower center',
                  bbox_to_anchor=(.5, -0.18),
                  frameon=True)

    plt.tight_layout(rect=[0, 0, 1, 0.9])

    safe_name = track_name.replace(" ", "_").replace("(", "").replace(")", "").replace(".mid", "")
    filename = f"segmentation_{safe_name}.pdf"
    plt.savefig(filename, format='pdf', bbox_inches='tight')
    plt.show()
    # -------------------
    # GLOBAL COMPACTNESS
    # -------------------
    global_compactness = compute_global_compactness(track_instruments, display_bands)
    print("Global compactness:", global_compactness)

    return intervals_by_inst, band_map, display_bands, track_instruments

# To use

## Functionals

This section defines the data preprocessing functions in order to analyze your pieces of music.

In [ ]:
def process_midi_separating_instruments(midi_file_path):
    midi_data = pretty_midi.PrettyMIDI(midi_file_path)
    tempo_changes, tempos = midi_data.get_tempo_changes()
    bpm = tempos[0] if len(tempos) > 0 else Fraction(120)
    seconds_to_beats = bpm / 60
    tatum = Fraction(1, (8 * 3))
    data = []

    for instrument in midi_data.instruments:
        instrument_name = instrument.name if instrument.name else 'Unknown'
        for note in instrument.notes:
            onset_in_beats = note.start * seconds_to_beats
            duration_in_beats = (note.end - note.start) * seconds_to_beats

            rounded_onset = round(onset_in_beats / tatum) * tatum
            rounded_duration = round(duration_in_beats / tatum) * tatum

            onset_value = float(rounded_onset) if rounded_onset.denominator % 3 != 0 else rounded_onset
            duration_value = float(rounded_duration) if rounded_duration.denominator % 3 != 0 else rounded_duration

            data.append({
                'midi_file': os.path.basename(midi_file_path),
                'instrument': instrument_name,
                'pitch': note.pitch,
                'onset': onset_value,
                'duration': duration_value
            })
    return data

def preprocess_midi_folder_to_dict(folder_path):
    dataset = {}

    for filename in os.listdir(folder_path):
        if filename.endswith(".mid") or filename.endswith(".midi"):
            file_path = os.path.join(folder_path, filename)
            try:
                midi_data = process_midi_separating_instruments(file_path)
                df = pd.DataFrame(midi_data)
                midi_file = df['midi_file'].iloc[0]

                if midi_file not in dataset:
                    dataset[midi_file] = {}

                for instrument, group in df.groupby('instrument'):
                    if instrument not in dataset[midi_file]:
                        dataset[midi_file][instrument] = []

                    for row in group.itertuples(index=False):
                        onset = Fraction(row.onset) if '/' in str(row.onset) else float(row.onset)
                        dataset[midi_file][instrument].append((onset, row.pitch))

            except Exception as e:
                print(f"❌ Error processing {filename}: {e}")
                continue
    return dataset

def get_subsets_with_points(preprocessed_dataset):
    excluded_tracks = {}

    subsets_data = {}
    for track_name, instruments_data in preprocessed_dataset.items():
        track_filename = track_name.split(" - ")[-1]
        for instrument_name, points in instruments_data.items():
            subset_name = f"{track_filename}_{instrument_name}"
            if subset_name not in excluded_tracks:
                subsets_data[subset_name] = points
    return subsets_data

## To complete by user

To analyze your own dataset.

In [ ]:
# 👇 To use:
midi_folder_path = "your_midi_folder_path_here"
preprocessed_dataset = preprocess_midi_folder_to_dict(midi_folder_path)
subsets_data = get_subsets_with_points(preprocessed_dataset)

print("✅ Dataset loaded with", len(subsets_data), "subsets.")

ground_truth_df = check_time_signature_presence(midi_folder_path)

# 1. Get all unique track names
track_names = {"_".join(k.split("_")[:-1]) for k in subsets_data}

# 2. Convert to sorted list
pieces_name = sorted(track_names)
print("Pieces name:", pieces_name)

# 3. Plot each unique piece
for piece in pieces_name:
    plot_metric_sections_by_best_compactness(subsets_data, piece, pattern_detection, dilation, ground_truth_df)